In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#Probably can`t be finished because of huge amount of data with kaggle hardware, add nrows parameter to run here
#Load Data
df1=pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")#,nrows = 50000
df2=pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

#import xgboost as xgb

df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2830743 entries, 0 to 692702
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max               

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [2]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    
    # Replace infinite values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # Calculate the mean of each numeric column, excluding NaNs
    column_means = df[numeric_cols].mean()
    
    # Replace NaN values with the mean of each numeric column
    df[numeric_cols] = df[numeric_cols].fillna(column_means)
    
    return df

In [3]:
df.dtypes

 Destination Port                 int64
 Flow Duration                    int64
 Total Fwd Packets                int64
 Total Backward Packets           int64
Total Length of Fwd Packets       int64
 Total Length of Bwd Packets      int64
 Fwd Packet Length Max            int64
 Fwd Packet Length Min            int64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max             int64
 Bwd Packet Length Min            int64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                     int64
 Flow IAT Min                     int64
Fwd IAT Total                     int64
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                      int64
 Fwd IAT Min                      int64


In [4]:
# Count the number of duplicate rows in the entire DataFrame
num_duplicates = df.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 308381


In [5]:
# Check if two columns are equal
column1 = ' Fwd Header Length'
column2 = ' Fwd Header Length.1'

are_columns_equal = df[column1].equals(df[column2])

if are_columns_equal:
    print(f"The columns '{column1}' and '{column2}' are equal.")
else:
    print(f"The columns '{column1}' and '{column2}' are not equal.")

The columns ' Fwd Header Length' and ' Fwd Header Length.1' are equal.


In [6]:
#Date preprocesing
df.drop([' Fwd Header Length.1'], axis=1, inplace=True)

In [7]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.head()

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes/s,flow_packets/s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets/s,bwd_packets/s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down/up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_avg_bytes/bulk,fwd_avg_packets/bulk,fwd_avg_bulk_rate,bwd_avg_bytes/bulk,bwd_avg_packets/bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:
df_cleaned = df.copy()
df_cleaned = clean_dataset(df_cleaned) # see methods at top of notebook

In [9]:
is_inf = np.isinf(df_cleaned.select_dtypes(include=['float64', 'int64'])).sum()
print("Number of infinite values:\n", is_inf)

Number of infinite values:
 destination_port               0
flow_duration                  0
total_fwd_packets              0
total_backward_packets         0
total_length_of_fwd_packets    0
total_length_of_bwd_packets    0
fwd_packet_length_max          0
fwd_packet_length_min          0
fwd_packet_length_mean         0
fwd_packet_length_std          0
bwd_packet_length_max          0
bwd_packet_length_min          0
bwd_packet_length_mean         0
bwd_packet_length_std          0
flow_bytes/s                   0
flow_packets/s                 0
flow_iat_mean                  0
flow_iat_std                   0
flow_iat_max                   0
flow_iat_min                   0
fwd_iat_total                  0
fwd_iat_mean                   0
fwd_iat_std                    0
fwd_iat_max                    0
fwd_iat_min                    0
bwd_iat_total                  0
bwd_iat_mean                   0
bwd_iat_std                    0
bwd_iat_max                    0
bwd_iat_min    

In [10]:
df_cleaned = df_cleaned.reset_index()
# Removing un-needed index column added by reset_index method
df_cleaned.drop('index', axis=1, inplace=True)

In [11]:
#Split dataset on train and test
from sklearn.model_selection import train_test_split
train, test=train_test_split(df_cleaned,test_size=0.3, random_state=10)

#Exploratory Analysis
# Descriptive statistics
train.describe()
test.describe()

# Packet Attack Distribution
train['label'].value_counts()
test['label'].value_counts()

label
BENIGN                        682092
DoS Hulk                       69066
PortScan                       47841
DDoS                           38346
DoS GoldenEye                   3041
FTP-Patator                     2359
SSH-Patator                     1786
DoS slowloris                   1764
DoS Slowhttptest                1649
Bot                              580
Web Attack � Brute Force         465
Web Attack � XSS                 203
Infiltration                      21
Web Attack � Sql Injection         8
Heartbleed                         2
Name: count, dtype: int64

In [12]:
#Scalling numerical attributes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# extract numerical attributes and scale it to have zero mean and unit variance  
cols = train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(train.select_dtypes(include=['float64','int64']))
sc_test = scaler.transform(test.select_dtypes(include=['float64','int64']))

# turn the result back to a dataframe
sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_testdf = pd.DataFrame(sc_test, columns = cols)

In [13]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 

# creating one hot encoder object 
onehotencoder = OneHotEncoder() 

trainDep = train['label'].values.reshape(-1,1)
trainDep = onehotencoder.fit_transform(trainDep).toarray()
testDep = test['label'].values.reshape(-1,1)
testDep = onehotencoder.transform(testDep).toarray()

In [14]:
train_X=sc_traindf
train_y=trainDep

test_X=sc_testdf
test_y=testDep

train_X.head()

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes/s,flow_packets/s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets/s,bwd_packets/s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down/up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_avg_bytes/bulk,fwd_avg_packets/bulk,fwd_avg_bulk_rate,bwd_avg_bytes/bulk,bwd_avg_packets/bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,-0.438658,-0.438762,-0.011304,-0.009496,-0.080510,-0.007054,-0.203022,0.716426,0.020199,-0.245273,-0.407513,0.537294,-0.376812,-0.399538,-0.057169,-0.278488,-0.281325,-0.363116,-0.374490,-0.044783,-0.431656,-0.274269,-0.339233,-0.368978,-0.118869,-0.344528,-0.203415,-0.236825,-0.27319,-0.116289,-0.220748,0.0,-0.010609,0.0,0.001131,0.001574,-0.257995,-0.183142,1.793738,-0.430276,-0.342719,-0.452480,-0.295317,-0.192195,-0.220748,-0.015743,-0.651342,-0.679755,-0.323409,-0.010609,-0.015791,0.466258,-0.274446,0.020199,-0.376812,0.0,0.0,0.0,0.0,0.0,0.0,-0.011304,-0.080510,-0.009496,-0.007054,-0.487262,-0.235181,-0.008575,0.002550,-0.123928,-0.102418,-0.147448,-0.099456,-0.352222,-0.109318,-0.357148,-0.339327
1,-0.438658,-0.439651,-0.009904,-0.008442,-0.080845,-0.006899,-0.247600,0.186825,-0.151639,-0.245273,-0.341261,2.411720,-0.163738,-0.399538,-0.001653,-0.230481,-0.288010,-0.363098,-0.375715,-0.055022,-0.431654,-0.274264,-0.339233,-0.368976,-0.118863,-0.344528,-0.203415,-0.236825,-0.27319,-0.116288,-0.220748,0.0,-0.010609,0.0,0.001134,0.001618,-0.233342,-0.022308,0.533937,-0.366702,-0.233195,-0.313678,-0.289661,-0.192195,-0.220748,-0.015743,-0.651342,-0.679755,-0.323409,-0.010609,-0.015791,0.466258,-0.199130,-0.151639,-0.163738,0.0,0.0,0.0,0.0,0.0,0.0,-0.009904,-0.080845,-0.008442,-0.006899,-0.487262,-0.235181,-0.006920,0.002569,-0.123928,-0.102418,-0.147448,-0.099456,-0.352222,-0.109318,-0.357148,-0.339327
2,1.773892,-0.439636,-0.009904,-0.010549,-0.090886,-0.007090,-0.289392,-0.309677,-0.312737,-0.245273,-0.447572,-0.596080,-0.505648,-0.399538,-0.057346,-0.269481,-0.287846,-0.363116,-0.375692,-0.054736,-0.431630,-0.274180,-0.339233,-0.368943,-0.118770,-0.344528,-0.203415,-0.236825,-0.27319,-0.116289,-0.220748,0.0,-0.010609,0.0,0.001133,0.001559,-0.248611,-0.184012,-0.647127,-0.468716,-0.563076,-0.467099,-0.295369,-0.192195,-0.220748,-0.015743,-0.651342,1.471119,-0.323409,-0.010609,-0.015791,-1.005983,-0.578721,-0.312737,-0.505648,0.0,0.0,0.0,0.0,0.0,0.0,-0.009904,-0.090886,-0.010549,-0.007090,-0.464044,-0.235181,-0.008575,0.002561,-0.123928,-0.102418,-0.147448,-0.099456,-0.352222,-0.109318,-0.357148,-0.339327
3,-0.437181,-0.282346,-0.008504,-0.009496,-0.090886,-0.007090,-0.289392,-0.309677,-0.312737,-0.245273,-0.447572,-0.596080,-0.505648,-0.399538,-0.057346,-0.278744,0.103172,-0.010511,-0.169838,-0.055001,-0.273975,0.003752,0.011212,-0.163669,-0.089026,-0.344528,-0.203415,-0.236825,-0.27319,-0.116289,-0.220748,0.0,-0.010609,0.0,0.001135,0.001588,-0.258126,-0.184007,-0.647127,-0.468716,-0.563076,-0.467099,-0.295369,-0.192195,-0.220748,-0.015743,1.535292,-0.679755,-0.323409,-0.010609,-0.015791,-1.005983,-0.578721,-0.312737,-0.505648,0.0,0.0,0.0,0.0,0.0,0.0,-0.008504,

In [15]:
# #Dataset Partition
# X_train,X_test,Y_train,Y_test = train_test_split(train_X,train_y,train_size=0.70, random_state=2)

#Fitting Models
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# # Train KNeighborsClassifier Model
# KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
# KNN_Classifier.fit(X_train, Y_train); 

# # Train LogisticRegression Model
# LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
# LGR_Classifier.fit(X_train, Y_train);

# # Train Gaussian Naive Baye Model
# BNB_Classifier = BernoulliNB()
# BNB_Classifier.fit(X_train, Y_train)

# # Train Decision Tree Model
# DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
# DTC_Classifier.fit(X_train, Y_train)

# Train Random Forest Classifier Model
RFC_Classifier = RandomForestClassifier(random_state=22)
# RFC_Classifier.fit(train_X, train_y)

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RFC_Classifier, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(train_X, train_y)
RFC_tuned = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [17]:
RFC_tuned.fit(train_X, train_y)

RandomForestClassifier(n_estimators=200, random_state=22)

In [21]:
import pickle
with open('trainedrfcmodel.pkl', 'wb') as f:
    pickle.dump(RFC_tuned,f)

In [23]:
with open('sc_ids.pkl','wb') as f:
    pickle.dump(scaler, f)

In [ ]:
#Evaluate Models
from sklearn import metrics

models = []
models.append(('RandomForestClassifier', RFC_tuned))

for i, v in models:
    scores = cross_val_score(v, train_X, train_y, cv=10)
    accuracy = metrics.accuracy_score(train_y, v.predict(train_X))
    confusion_matrix = metrics.multilabel_confusion_matrix(train_y, v.predict(train_X))
    classification = metrics.classification_report(train_y, v.predict(train_X))
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()

In [20]:
#Validate Models
from sklearn import metrics

for i, v in models:
    accuracy = metrics.accuracy_score(test_y, v.predict(test_X))
    confusion_matrix = metrics.multilabel_confusion_matrix(test_y, v.predict(test_X))
    classification = metrics.classification_report(test_y, v.predict(test_X))
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()        


============================== RandomForestClassifier Model Test Results ==============================

Model Accuracy:
 0.9986517086795812

Confusion matrix:
 [[[166800    331]
  [   509 681583]]

 [[848599     44]
  [   171    409]]

 [[810873      4]
  [    19  38327]]

 [[846174      8]
  [     9   3032]]

 [[780012    145]
  [    98  68968]]

 [[847563     11]
  [    13   1636]]

 [[847453      6]
  [    13   1751]]

 [[846864      0]
  [     2   2357]]

 [[849221      0]
  [     0      2]]

 [[849202      0]
  [    14      7]]

 [[801067    315]
  [    21  47820]]

 [[847437      0]
  [     6   1780]]

 [[848614    144]
  [    83    382]]

 [[849215      0]
  [     8      0]]

 [[848993     27]
  [   179     24]]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    682092
           1       0.90      0.71      0.79       580
           2       1.00      1.00      1.00     38346
           3       1.00   